# Get the red and white species

In [98]:
import pandas as pd
import numpy as np
import toyplot
from dateutil import parser
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import imageio
import os
from IPython.display import Image
import rasterio

In [2]:
%pwd

'/Users/patrickmckenzie/googledrive/projects/flower_color_phenology/notebooks'

In [3]:
%cd ..

/Users/patrickmckenzie/googledrive/projects/flower_color_phenology


In [11]:
coldf = pd.read_csv('./data/cleaned_matched_colors.csv',index_col=0)

In [18]:
inatdata = pd.read_csv('./data/combined_raw_inaturalist_export.csv')

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_7300/86142954.py:1: DtypeWarning: Columns (0,5,14,18,19,20,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  inatdata = pd.read_csv('./data/combined_raw_inaturalist_export.csv')


In [19]:
# let's make a column for the first two words of every scientific name in the data
inatdata['binomial'] = [' '.join(str(i).split( )[:2]) for i in inatdata.scientific_name]

In [28]:
color_list = []
for obs_sp in list(inatdata.binomial):
    subrows = coldf[coldf.species_name.eq(obs_sp.lower())]
    if len(subrows):
        color = np.random.choice(subrows.color).lower()
    else:
        color = np.nan
    color_list.append(color)

In [31]:
inatdata['color'] = color_list

In [36]:
inatdata_plus_color = inatdata[~inatdata.color.isna()]

In [37]:
len(inatdata_plus_color.index)

673100

In [44]:
days_list = []
for date in inatdata_plus_color.observed_on:
    dt = parser.parse(date)
    day_of_year = dt.timetuple().tm_yday
    days_list.append(day_of_year)
    
inatdata_plus_color['day_of_year'] = days_list
inatdata_plus_color

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_7300/4150388654.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inatdata_plus_color['day_of_year'] = days_list


,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,positioning_method,positioning_device,species_guess,scientific_name,common_name,iconic_taxon_name,taxon_id,binomial,color,day_of_year
5,38456,"November 4, 2011 8:01:47 AM CDT",2011-11-04,2011-11-04 13:01:47 UTC,Central Time (US & Canada),2281,cullen,Cullen Hanks,2011-11-07 17:21:27 UTC,2023-04-20 22:36:31 UTC,...,NaN,NaN,Violet Twining Snapdragon,Maurandella antirrhiniflora,Snapdragon Vine,Plantae,77967.0,Maurandella antirrhiniflora,purple,308
10,49042,2010-07-15,2010-07-15,NaN,Pacific Time (US & Canada),2991,sea-kangaroo,NaN,2012-01-26 22:27:31 UTC,2023-03-05 17:35:02 UTC,...,NaN,NaN,Showy Milkweed,Asclepias speciosa,showy milkweed,Plantae,62292.0,Asclepias speciosa,white,196
12,58974,"March 19, 2012 18:04",2012-03-19,2012-03-19 23:04:00 UTC,Central Time (US & Canada),5107,mikegras,"Michael J. Gras, M.Ed.",2012-03-20 01:16:11 UTC,2021-10-28 04:13:57 UTC,...,NaN,NaN,flowering dogwood,Cornus florida,flowering dogwood,Plantae,54777.0,Cornus florida,white,79
14,60348,"March 26, 2012 19:31",2012-03-26,2012-03-27 00:31:00 UTC,Central Time (US & Canada),5429,gaberlunzi,NaN,2012-03-27 13:28:30 UTC,2020-11-28 13:02:20 UTC,...,NaN,NaN,Cleavers,Galium aparine,catchweed bedstraw,Plantae,53059.0,Galium aparine,white,86
21,75169,2012-05-05,2012-05-05,NaN,Central Time (US & Canada),6437,crystalbrooks2012,NaN,2012-05-06 02:19:49 UTC,2017-09-25 17:39:59 UTC,...,NaN,NaN,Persian Silk Tree,Albizia julibrissin,Persian silk tree,Plantae,47443.0,Albizia julibrissin,yellow,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763871,187058444,2023/10/10 4:30 PM,2023-10-10,2023-10-10 23:30:00 UTC,Pacific Time (US & Canada),1878202,rudester,Rudy,2023-10-10 23:30:28 UTC,2023-10-11 21:35:05 UTC,...,NaN,NaN,tree tobacco,Nicotiana glauca,tree tobacco,Plantae,58334.0,Nicotiana glauca,orange,283
1763875,187086683,2023-10-10 17:48:01-07:00,2023-10-10,2023-10-11 00:48:01 UTC,Pacific Time (US & Canada),1741810,hsug1747,NaN,2023-10-11 05:31:53 UTC,2023-10-11 17:29:40 UTC,...,NaN,NaN,tree tobacco,Nicotiana glauca,tree tobacco,Plantae,58334.0,Nicotiana glauca,orange,283
1763879,187152623,2023-10-11 10:08:36-07:00,2023-10-11,2023-10-11 17:08:36 UTC,Pacific Time (US & Canada),573539,bmacm,Barbara Millett,2023-10-11 19:08:46 UTC,2023-10-11 22:46:44 UTC,...,NaN,NaN,scarlet monkeyflower,Erythranthe cardinalis,scarlet monkeyflower,Plantae,319974.0,Erythranthe cardinalis,red,284
1763880,187155066,2023-10-11 12:30:00-07:00,2023-10-11,2023-10-11 19:30:00 UTC,Pacific Time (US & Canada),7396362,noahhaas,NaN,2023-10-11 19:30:21 UTC,2023-10-12 03:10:47 UTC,...,NaN,NaN,mule fat,Baccharis salicifolia,mule fat,Plantae,57913.0,Baccharis salicifolia,yellow,284


In [45]:
obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(inatdata_plus_color.day_of_year.eq(day_))
    obs_by_day.append(total_obs_day)

In [52]:
toyplot.scatterplot(obs_by_day);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td19defb1bb2547978d9f7dc3261ad57b"> 0 100 200 300 400 0 5000 10000 15000 20000

In [50]:
subdf = inatdata_plus_color[inatdata_plus_color.color.eq('red')]
red_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    red_obs_by_day.append(total_obs_day)

In [51]:
subdf = inatdata_plus_color[inatdata_plus_color.color.eq('white')]
white_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    white_obs_by_day.append(total_obs_day)

In [55]:
# show the occurrences of red vs white flowers by day of year
canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(xlabel = 'day of year',ylabel='num occurrences of flower color')
mark3 = axes.scatterplot(white_obs_by_day,size=10,color='black')
mark1 = axes.scatterplot(red_obs_by_day,size=10,color='red')

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0fe85a1e70d443b19a7bb4955240781c"> 0 100 200 300 400 day of year 0 2000 4000 6000 num occurrences of flower color

# Make a rough scatter map

In [71]:
map_dir = './red_white_rough'
if not os.path.exists(map_dir):
    os.mkdir(map_dir)

In [72]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
#gdf = GeoDataFrame(subdf, geometry=geometry)   
polygon = Polygon([(-130, 24), (-130, 54), (-59, 54), (-59, 24), (-130, 24)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=world.crs)

world_clipped = world.clip(polygon)

In [ ]:
for color1 in ['red',
]:
    #color1 = 'red'
    color2 = 'white'

    for starter_val in range(0,351):
        end_val = starter_val + 15

        fig, axs = plt.subplots(ncols=2,nrows=1, gridspec_kw=dict(width_ratios=[1,1]),figsize=(20,6))
        fig.suptitle('Days ' + str(starter_val) + '-' + str(starter_val+15),fontsize=20)

        subdf = inatdata_plus_color[inatdata_plus_color.color.eq(color1)]
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[0],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);

        axs[0].set_title(color1.capitalize() + " Flowers", fontsize=20)

        subdf = inatdata_plus_color[inatdata_plus_color.color.eq(color2)]
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[1],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);
        axs[1].set_title(color2.capitalize() +" Flowers", fontsize=20)

        plt.savefig(os.path.join(map_dir,'rough'+str(starter_val)+'.png'))

    filenames = [os.path.join(map_dir,'rough'+str(i) +'.png') for i in range(0,351)]

    with imageio.get_writer(os.path.join(map_dir,'rough_'+color1+'_'+color2+'.gif'), mode='I',duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

# Now try the same thing with random colors, but the size of the red flower dataset

In [67]:
numred = np.sum(inatdata_plus_color.color.eq('red'))
sample_df = inatdata_plus_color.sample(numred)

In [68]:
map_dir = './random_white_rough'
if not os.path.exists(map_dir):
    os.mkdir(map_dir)

In [69]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
#gdf = GeoDataFrame(subdf, geometry=geometry)   
polygon = Polygon([(-130, 24), (-130, 54), (-59, 54), (-59, 24), (-130, 24)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=world.crs)

world_clipped = world.clip(polygon)

In [ ]:
for color1 in ['random',
]:
    #color1 = 'red'
    color2 = 'white'

    for starter_val in range(0,351):
        end_val = starter_val + 15

        fig, axs = plt.subplots(ncols=2,nrows=1, gridspec_kw=dict(width_ratios=[1,1]),figsize=(20,6))
        fig.suptitle('Days ' + str(starter_val) + '-' + str(starter_val+15),fontsize=20)

        subdf = sample_df
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[0],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);

        axs[0].set_title(color1.capitalize() + " Flowers", fontsize=20)

        subdf = inatdata_plus_color[inatdata_plus_color.color.eq(color2)]
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[1],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);
        axs[1].set_title(color2.capitalize() +" Flowers", fontsize=20)

        plt.savefig(os.path.join(map_dir,'rough'+str(starter_val)+'.png'))

    filenames = [os.path.join(map_dir,'rough'+str(i) +'.png') for i in range(0,351)]

    with imageio.get_writer(os.path.join(map_dir,'rough_'+color1+'_'+color2+'.gif'), mode='I',duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

### This gif doesn't show white leading in spring, suggesting there's more to the red vs. white thing than just mismatch in number of observations.

# Maxent running

In [95]:
class Mapper:
    """
    The object central to `smood` (simple mapping of occurrence data).
    """
    def __init__(
        self,
        df,
        run_name="test",
        lat_range=None,
        lon_range=None,
        worldclim_layers=list(range(1, 20)),
        outputs_dir="maxent_outputs/",
        write_outputs=False,
        maxent_path=None,
        worldclim_dir=None,
        ):
        """
        The object central to `smood` (simple mapping of occurrence data).
        Users supply a species name, latitude range, and longitude range, and
        then they can run automated maxent sdms over this.
        Parameters:
        -----------
        species_name (str):
            The name of the species.
            e.g., "Monarda fistulosa"
        lat_range (list, tuple):
            A list of the latitude values, low and high, used as bounds for the map.
            e.g., (30, 50)
            Values must be from the range [-90,90]
            These values are sorted later on, so the order doesn't matter.
        lon_range (list, tuple):
            A list of the longitude values, low and high, used as bounds for the map.
            e.g., (-100, -50)
            Values must be from the range [-180,180]
            These values are sorted later on, so the order doesn't matter.
        worldclim_layers (list):
            A list of the layers to use from worldclim. By default, this list contains
            integers 1 through 19, corresponding to all 19 worldclime layers.
        """

        self.profile = {}

        self.df = df
        
        self.profile['run_name'] = run_name
        if lat_range:
            _ = np.sort(lat_range)
            self.profile['ymin'] = _[0]
            self.profile['ymax'] = _[1]
        else:
            self.profile['ymin'] = None
            self.profile['ymax'] = None

        if lon_range:
            _ = np.sort(lon_range)
            self.profile['xmin'] = _[0]
            self.profile['xmax'] = _[1]
        else:
            self.profile['xmin'] = None
            self.profile['xmax'] = None

        if not maxent_path:
            # make the maxent path give the path to the .jar in the package directory...
            self.maxent_path = os.path.join(self.upper_package_level,
                                            'bins',
                                            'maxent.jar')
        else:
            self.maxent_path = maxent_path

        if not worldclim_dir:
            self.worldclim_dir = os.path.join(self.upper_package_level,
                                              'worldclim')
        else:
            self.worldclim_dir = worldclim_dir

        self.worldclim_dict = {
            1:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_01.tif"),
            2:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_02.tif"),
            3:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_03.tif"),
            4:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_04.tif"),
            5:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_05.tif"),
            6:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_06.tif"),
            7:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_07.tif"),
            8:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_08.tif"),
            9:  os.path.join(self.worldclim_dir, "wc2.0_bio_10m_09.tif"),
            10: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_10.tif"),
            11: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_11.tif"),
            12: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_12.tif"),
            13: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_13.tif"),
            14: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_14.tif"),
            15: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_15.tif"),
            16: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_16.tif"),
            17: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_17.tif"),
            18: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_18.tif"),
            19: os.path.join(self.worldclim_dir, "wc2.0_bio_10m_19.tif"),
        }

        if worldclim_layers:
            self.profile['worldclim_layers'] = worldclim_layers

        # name folder for maxent outputs
        self.outputs_dir = outputs_dir

        # name folder for clipped/converted worldclim
        #self.envfiles_dir = os.path.join(self.outputs_dir,
        #                                 "envfiles")
        self.envfiles_dir = os.path.join(self.outputs_dir,
                                         "envfiles")

        self.key = None
        self.write_outputs = write_outputs

    def _get_gbif_occs(self):
        # get the gbif key for our species
        self.occfile = os.path.join(self.outputs_dir,
                                    self.profile['run_name']+".csv")

        # make lists to fill
        self.lats = list(self.df.latitude)
        self.lons = list(self.df.longitude)

        # prepare array to write to csv
        csvarr = np.vstack([np.repeat(self.profile['run_name'], len(self.lons)),
                            self.lons,
                            ["{}{}".format(a_, b_) for a_, b_ in zip(self.lats, 
                                                                     np.repeat('\n', 
                                                                               len(self.lats)
                                                                               )
                                                                     )
                             ]
                            ]).T
        # write occurrence data to csv
        with open(self.occfile, 'w') as f:
            f.write('Species,Longitude,Latitude\n')
            for line in csvarr:
                f.write(",".join(line))

        # make these easier to work with downstream
        self.lons = np.array(self.lons)
        self.lats = np.array(self.lats)

    def _write_env_rasters(self):
        """
        Looks at raw worldclim data, clips it to specified bounding box, and writes the result as ascii.
        """
        # loop through the worldclim master files
        for idx, filepath in enumerate([self.worldclim_dict[layer_int] for layer_int in self.profile['worldclim_layers']]):
            # open with rasterio
            envdata = rasterio.open(filepath, 'r')

            # define a window using lat and lon
            win1 = envdata.window(self.profile['xmin'],
                                  self.profile['ymin'],
                                  self.profile['xmax'],
                                  self.profile['ymax'])

            # read env data from the window
            windowarr = envdata.read(window=win1)[0]

            # get affine transform (this will be used to get cell size)
            aff = envdata.profile['transform']

            # get number of columns in the window
            ncols = windowarr.shape[1]

            # get number of rows in the window
            nrows = windowarr.shape[0]

            # define the lower left corner x coordinate (in degrees)
            xllcorner = self.profile['xmin']

            # define the lower left corner y coordinate (in degrees)
            yllcorner = self.profile['ymin']

            # save the cell size from the affine transform
            cellsize = aff.a

            # record the value corresponding to nodata
            nodata_value = envdata.profile['nodata']

            # save ascii file -- saving array with space delimiter, and metadata as a header
            np.savetxt(os.path.join(self.envfiles_dir,
                                    str(idx)+'.asc'),
                       windowarr,
                       delimiter=' ',
                       comments='',
                       header="".join(['ncols {}\n'.format(ncols),
                                       'nrows {}\n'.format(nrows),
                                       'xllcorner {}\n'.format(xllcorner),
                                       'yllcorner {}\n'.format(yllcorner),
                                       'cellsize {}\n'.format(cellsize),
                                       'nodata_value {}'.format(nodata_value)]))

    def run(self):
        """
        Runs gbif and maxent on the species name and bounds provided.
        """

        # make these directories
        #os.mkdir(self.outputs_dir)
        #os.mkdir(self.envfiles_dir)

        self._get_gbif_occs()
        self._write_env_rasters()

        # run maxent from command line

        mkseq = Maxent(self.maxent_path)
        mkseq.open_subprocess()
        mkseq.feed_maxent(self.envfiles_dir,
                          self.occfile,
                          self.outputs_dir,
                          )
        mkseq.close_subprocess()

        # save png output from maxent
        self.maxent_image = Image(os.path.join(self.outputs_dir,
                                               "plots",
                                               self.profile['run_name']+".png"))

        # save raster output from maxent
        self.density_mat = np.genfromtxt(os.path.join(self.outputs_dir,
                                                      self.profile['run_name']+".asc"),
                                         delimiter=' ',
                                         skip_header=6)
        # remove the nans (maxent saves these as -9999)
        self.density_mat[self.density_mat == -9999] = np.nan

        # remove the outputs, we have what we need in memory
        if not self.write_outputs:
            rmtree(self.outputs_dir)
            
import os
import subprocess as sps


class Maxent:
    """
    Opens a view to seq-gen in a subprocess so that many gene trees can be
    cycled through without the overhead of opening/closing subprocesses.
    """

    def __init__(self,
                 maxent_path):

        # set binary path for conda env and check for binary
        self.binary = maxent_path
        assert os.path.exists(self.binary), (
            "binary {} not found".format(self.binary))

        # call open_subprocess to set the shell
        self.shell = None

    def open_subprocess(self):
        """
        Open a persistent Popen bash shell
        """
        # open
        self.shell = sps.Popen(
            ["bash"], stdin=sps.PIPE, stdout=sps.PIPE, bufsize=0)

    def close_subprocess(self):
        """
        Cleanup and shutdown the subprocess shell.
        """
        self.shell.stdin.close()
        self.shell.terminate()
        self.shell.wait(timeout=1.0)

    def feed_maxent(self, envfiles_dir, occfile, outputs_dir):
        """
        Feed a command string a read results until empty line.
        TODO: allow kwargs to add additional seq-gen args.
        """
        # command string
        cmd = (
            "java -mx512m -jar {} nowarnings environmentallayers={} samplesfile={} outputdirectory={} redoifexists autorun; echo done\n"
            .format(self.binary, envfiles_dir, occfile, outputs_dir)
        )

        # feed to the shell
        self.shell.stdin.write(cmd.encode())
        self.shell.stdin.flush()

        # catch returned results until done\n
        hold = []
        for line in iter(self.shell.stdout.readline, b"done\n"):
            hold.append(line.decode())

In [ ]:
# do all of the reds
reds = ['red']
for start_day in range(0,350,1):
    subdf = inatdata_plus_color[inatdata_plus_color.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(reds)]
    mapper = Mapper(subdf,run_name='red'+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='./data/maxent/outputs',
       maxent_path='./bins/maxent.jar',
       worldclim_dir='./data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

In [ ]:
# do all of the reds
colors = ['white']
run_name_prefix = 'white'
for start_day in range(0,350,1):
    subdf = inatdata_plus_color[inatdata_plus_color.day_of_year.isin(range(start_day,start_day+15))]
    subdf = subdf[subdf.color.isin(colors)]
    mapper = Mapper(subdf,run_name=run_name_prefix+'_'+str(start_day),lat_range=[24,54],lon_range=[-130,-59],
       outputs_dir='./data/maxent/outputs',
       maxent_path='./bins/maxent.jar',
       worldclim_dir='./data/worldclim/',write_outputs=True,
        worldclim_layers=[1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19]
      )
    mapper.run()

# Redo with the gpt-labeled things:

In [1]:
import pandas as pd
import numpy as np
import toyplot
from dateutil import parser
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import imageio
import os
from IPython.display import Image
import rasterio

In [2]:
%pwd

'/Users/patrickmckenzie/googledrive/projects/flower_color_phenology/notebooks'

In [3]:
%cd ..

/Users/patrickmckenzie/googledrive/projects/flower_color_phenology


In [4]:
inatdata = pd.read_csv('./data/GPT_plus_TRY_combined_raw_inaturalist_export_nafiltered.csv')

/var/folders/qh/c7rwx84921n2kznn09zb2pnh0000gn/T/ipykernel_29968/3780635703.py:1: DtypeWarning: Columns (14,29) have mixed types. Specify dtype option on import or set low_memory=False.
  inatdata = pd.read_csv('./data/GPT_plus_TRY_combined_raw_inaturalist_export_nafiltered.csv')


In [5]:
inatdata.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,coordinates_obscured,positioning_method,positioning_device,species_guess,scientific_name,common_name,iconic_taxon_name,taxon_id,binomial,color
0,47,"March 21, 2008",2008-03-21,NaN,Central Time (US & Canada),7,lisa_and_robb,NaN,2008-03-25 11:21:54 UTC,2023-03-12 05:50:05 UTC,...,False,NaN,NaN,Texas Bluebonnet,Lupinus texensis,Texas bluebonnet,Plantae,49564.0,Lupinus texensis,blue
1,8009,2010-07-20,2010-07-20,NaN,Mountain Time (US & Canada),453,leighannemcc,NaN,2010-08-06 19:30:15 UTC,2020-06-04 22:16:05 UTC,...,False,NaN,NaN,Solanum rostratum,Solanum rostratum,buffalo-bur,Plantae,62642.0,Solanum rostratum,yellow
2,32167,"September 22, 2011 10:58",2011-09-22,2011-09-22 14:58:00 UTC,Eastern Time (US & Canada),2370,ctracey,Christopher Tracey,2011-09-23 03:14:29 UTC,2022-05-17 18:47:54 UTC,...,False,NaN,NaN,Fendler's sandmat,Euphorbia fendleri,Fendler's Sandmat,Plantae,148407.0,Euphorbia fendleri,yellow
3,35715,"October 14, 2011 2:17:52 PM CDT",2011-10-14,2011-10-14 19:17:52 UTC,Central Time (US & Canada),2281,cullen,Cullen Hanks,2011-10-15 04:38:10 UTC,2022-05-12 15:24:40 UTC,...,False,NaN,NaN,Spigelia texana,Spigelia texana,Texas pinkroot,Plantae,520535.0,Spigelia texana,pink
4,38015,"March 13, 2011 15:36",2011-03-13,2011-03-13 20:36:00 UTC,Central Time (US & Canada),2670,atxnaturalist,Kari Gaukler,2011-11-03 03:55:14 UTC,2020-11-22 14:54:48 UTC,...,False,NaN,NaN,Texas baby blue eyes,Nemophila phacelioides,Texas baby blue eyes,Plantae,120550.0,Nemophila phacelioides,blue


In [6]:
days_list = []
for date in inatdata.observed_on:
    dt = parser.parse(date)
    day_of_year = dt.timetuple().tm_yday
    days_list.append(day_of_year)
    
inatdata['day_of_year'] = days_list
inatdata

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,positioning_method,positioning_device,species_guess,scientific_name,common_name,iconic_taxon_name,taxon_id,binomial,color,day_of_year
0,47,"March 21, 2008",2008-03-21,NaN,Central Time (US & Canada),7,lisa_and_robb,NaN,2008-03-25 11:21:54 UTC,2023-03-12 05:50:05 UTC,...,NaN,NaN,Texas Bluebonnet,Lupinus texensis,Texas bluebonnet,Plantae,49564.0,Lupinus texensis,blue,81
1,8009,2010-07-20,2010-07-20,NaN,Mountain Time (US & Canada),453,leighannemcc,NaN,2010-08-06 19:30:15 UTC,2020-06-04 22:16:05 UTC,...,NaN,NaN,Solanum rostratum,Solanum rostratum,buffalo-bur,Plantae,62642.0,Solanum rostratum,yellow,201
2,32167,"September 22, 2011 10:58",2011-09-22,2011-09-22 14:58:00 UTC,Eastern Time (US & Canada),2370,ctracey,Christopher Tracey,2011-09-23 03:14:29 UTC,2022-05-17 18:47:54 UTC,...,NaN,NaN,Fendler's sandmat,Euphorbia fendleri,Fendler's Sandmat,Plantae,148407.0,Euphorbia fendleri,yellow,265
3,35715,"October 14, 2011 2:17:52 PM CDT",2011-10-14,2011-10-14 19:17:52 UTC,Central Time (US & Canada),2281,cullen,Cullen Hanks,2011-10-15 04:38:10 UTC,2022-05-12 15:24:40 UTC,...,NaN,NaN,Spigelia texana,Spigelia texana,Texas pinkroot,Plantae,520535.0,Spigelia texana,pink,287
4,38015,"March 13, 2011 15:36",2011-03-13,2011-03-13 20:36:00 UTC,Central Time (US & Canada),2670,atxnaturalist,Kari Gaukler,2011-11-03 03:55:14 UTC,2020-11-22 14:54:48 UTC,...,NaN,NaN,Texas baby blue eyes,Nemophila phacelioides,Texas baby blue eyes,Plantae,120550.0,Nemophila phacelioides,blue,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719132,187151184,2023-10-11 10:18:05-07:00,2023-10-11,2023-10-11 17:18:05 UTC,Pacific Time (US & Canada),573539,bmacm,Barbara Millett,2023-10-11 18:55:36 UTC,2023-10-11 22:10:28 UTC,...,NaN,NaN,greenspot nightshade,Solanum douglasii,greenspot nightshade,Plantae,64105.0,Solanum douglasii,white,284
1719133,187152623,2023-10-11 10:08:36-07:00,2023-10-11,2023-10-11 17:08:36 UTC,Pacific Time (US & Canada),573539,bmacm,Barbara Millett,2023-10-11 19:08:46 UTC,2023-10-11 22:46:44 UTC,...,NaN,NaN,scarlet monkeyflower,Erythranthe cardinalis,scarlet monkeyflower,Plantae,319974.0,Erythranthe cardinalis,red,284
1719134,187155066,2023-10-11 12:30:00-07:00,2023-10-11,2023-10-11 19:30:00 UTC,Pacific Time (US & Canada),7396362,noahhaas,NaN,2023-10-11 19:30:21 UTC,2023-10-12 03:10:47 UTC,...,NaN,NaN,mule fat,Baccharis salicifolia,mule fat,Plantae,57913.0,Baccharis salicifolia,yellow,284
1719135,187157269,2023-10-11 10:15:01-07:00,2023-10-11,2023-10-11 17:15:01 UTC,Pacific Time (US & Canada),573539,bmacm,Barbara Millett,2023-10-11 19:48:42 UTC,2023-10-11 20:27:06 UTC,...,NaN,NaN,White Sweetclover,Melilotus albus,White Sweetclover,Plantae,58907.0,Melilotus albus,white,284


In [8]:
obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(inatdata.day_of_year.eq(day_))
    obs_by_day.append(total_obs_day)

In [9]:
toyplot.scatterplot(obs_by_day);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5241cace8b8a4e1892f650f698fb0f08"> 0 100 200 300 400 0 20000 40000 60000

In [11]:
subdf = inatdata[inatdata.color.eq('red')]
red_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    red_obs_by_day.append(total_obs_day)

In [12]:
subdf = inatdata[inatdata.color.eq('white')]
white_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    white_obs_by_day.append(total_obs_day)

In [13]:
# show the occurrences of red vs white flowers by day of year
canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(xlabel = 'day of year',ylabel='num occurrences of flower color')
mark3 = axes.scatterplot(white_obs_by_day,size=10,color='black')
mark1 = axes.scatterplot(red_obs_by_day,size=10,color='red')

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tce93218765214431a078b929c4939c24"> 0 100 200 300 400 day of year 0 5000 10000 15000 num occurrences of flower color

In [46]:
subdf = inatdata[inatdata.color.eq('white')].sample(len(inatdata[inatdata.color.eq('red')].index))
white_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    white_obs_by_day.append(total_obs_day)

In [47]:
subdf = inatdata[inatdata.color.eq('red')]
red_obs_by_day=[]
for day in range(365-3):
    dayrange= range(day,day+3)
    total_obs_day = 0
    for day_ in dayrange:
        total_obs_day += np.sum(subdf.day_of_year.eq(day_))
    red_obs_by_day.append(total_obs_day)

In [48]:
# show the occurrences of red vs white flowers by day of year
canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(xlabel = 'day of year',ylabel='num occurrences of flower color')
mark3 = axes.scatterplot(white_obs_by_day,size=10,color='black')
mark1 = axes.scatterplot(red_obs_by_day,size=10,color='red')

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te9efcd4cb8004b0a846eb4df2f6fb461"> 0 100 200 300 400 day of year 0 1000 2000 3000 num occurrences of flower color

# Eek!!! Suddenly these don't look too different!

In [50]:
map_dir = './red_white_gpt_rough'
if not os.path.exists(map_dir):
    os.mkdir(map_dir)

In [51]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
#gdf = GeoDataFrame(subdf, geometry=geometry)   
polygon = Polygon([(-130, 24), (-130, 54), (-59, 54), (-59, 24), (-130, 24)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=world.crs)

world_clipped = world.clip(polygon)

In [ ]:
for color1 in ['red',
]:
    #color1 = 'red'
    color2 = 'white'

    for starter_val in range(0,351):
        end_val = starter_val + 15

        fig, axs = plt.subplots(ncols=2,nrows=1, gridspec_kw=dict(width_ratios=[1,1]),figsize=(20,6))
        fig.suptitle('Days ' + str(starter_val) + '-' + str(starter_val+15),fontsize=20)

        subdf = inatdata[inatdata.color.eq(color1)]
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[0],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);

        axs[0].set_title(color1.capitalize() + " Flowers", fontsize=20)

        subdf = inatdata[inatdata.color.eq(color2)]
        subdf = subdf[subdf.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[1],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);
        axs[1].set_title(color2.capitalize() +" Flowers", fontsize=20)

        plt.savefig(os.path.join(map_dir,'rough'+str(starter_val)+'.png'))

    filenames = [os.path.join(map_dir,'rough'+str(i) +'.png') for i in range(0,351)]

    with imageio.get_writer(os.path.join(map_dir,'rough_'+color1+'_'+color2+'.gif'), mode='I',duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
map_dir = './red_subsamp_white_gpt_rough'
if not os.path.exists(map_dir):
    os.mkdir(map_dir)

for color1 in ['red',
]:
    #color1 = 'red'
    subdf_color1 = inatdata[inatdata.color.eq(color1)]
    color2 = 'white'
    subdf_color2 = inatdata[inatdata.color.eq(color2)].sample(len(subdf_color1.index))
    
    for starter_val in range(0,351):
        end_val = starter_val + 15

        fig, axs = plt.subplots(ncols=2,nrows=1, gridspec_kw=dict(width_ratios=[1,1]),figsize=(20,6))
        fig.suptitle('Days ' + str(starter_val) + '-' + str(starter_val+15),fontsize=20)

        subdf = subdf_color1[subdf_color1.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[0],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);

        axs[0].set_title(color1.capitalize() + " Flowers", fontsize=20)

        subdf = subdf_color2[subdf_color2.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[1],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);
        axs[1].set_title(color2.capitalize() +" Flowers", fontsize=20)

        plt.savefig(os.path.join(map_dir,'rough'+str(starter_val)+'.png'))

    filenames = [os.path.join(map_dir,'rough'+str(i) +'.png') for i in range(0,351)]

    with imageio.get_writer(os.path.join(map_dir,'rough_'+color1+'_'+color2+'.gif'), mode='I',duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
map_dir = './red_subsamp_blue_gpt_rough'
if not os.path.exists(map_dir):
    os.mkdir(map_dir)

for color1 in ['red',
]:
    #color1 = 'red'
    subdf_color1 = inatdata[inatdata.color.eq(color1)]
    color2 = 'blue'
    subdf_color2 = inatdata[inatdata.color.eq(color2)].sample(len(subdf_color1.index))
    
    for starter_val in range(0,351):
        end_val = starter_val + 15

        fig, axs = plt.subplots(ncols=2,nrows=1, gridspec_kw=dict(width_ratios=[1,1]),figsize=(20,6))
        fig.suptitle('Days ' + str(starter_val) + '-' + str(starter_val+15),fontsize=20)

        subdf = subdf_color1[subdf_color1.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[0],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);

        axs[0].set_title(color1.capitalize() + " Flowers", fontsize=20)

        subdf = subdf_color2[subdf_color2.day_of_year.isin(list(range(starter_val, end_val)))]

        geometry = [Point(xy) for xy in zip(subdf['longitude'], subdf['latitude'])]
        gdf = GeoDataFrame(subdf, geometry=geometry)   

        gdf.plot(ax=world_clipped.plot(ax=axs[1],figsize=(10, 6)), marker='o', color='red', markersize=15,aspect=1.3);
        axs[1].set_title(color2.capitalize() +" Flowers", fontsize=20)

        plt.savefig(os.path.join(map_dir,'rough'+str(starter_val)+'.png'))

    filenames = [os.path.join(map_dir,'rough'+str(i) +'.png') for i in range(0,351)]

    with imageio.get_writer(os.path.join(map_dir,'rough_'+color1+'_'+color2+'.gif'), mode='I',duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

In [72]:
np.random.choice(np.unique(inatdata[inatdata.color.eq('red')].scientific_name),20)

array(['Papaver rhoeas', 'Cosmos bipinnatus',
       'Cirsium arizonicum tenuisectum', 'Clintonia andrewsiana',
       'Scirpus pallidus', 'Trillium erectum',
       'Keckiella rothrockii jacintensis', 'Hamelia patens',
       'Lysimachia monelli', 'Diplacus nanus',
       'Penstemon eatonii exsertus', 'Salvia gesneriiflora',
       'Dianthus deltoides deltoides', 'Aronia arbutifolia',
       'Spondias purpurea', 'Pistacia chinensis', 'Melinis repens',
       'Penstemon miniatus', 'Decodon verticillatus',
       'Centaurium pulchellum'], dtype=object)

# Make sliced historgrams of red vs white